In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,San Quintin,MX,2020-08-01 19:08:18,30.48,-115.95,80.20,55,4,9.78
1,1,Atuona,PF,2020-08-01 19:06:54,-9.80,-139.03,79.30,75,99,17.36
2,2,Punta Arenas,CL,2020-08-01 19:06:58,-53.15,-70.92,39.20,86,75,8.05
3,3,Yellowknife,CA,2020-08-01 19:06:54,62.46,-114.35,77.00,64,75,12.75
4,4,Raudeberg,NO,2020-08-01 19:10:22,61.99,5.14,64.99,77,100,12.75


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds , dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind , dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 60


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
14,14,Hobart,AU,2020-08-01 19:05:32,-42.88,147.33,50.00,61,3,5.82
16,16,Masty,BY,2020-08-01 19:10:23,53.41,24.54,57.79,82,0,6.64
20,20,New Norfolk,AU,2020-08-01 19:07:10,-42.78,147.06,48.99,49,0,4.00
21,21,Mount Gambier,AU,2020-08-01 19:07:15,-37.83,140.77,46.40,93,90,2.13
22,22,Bluff,NZ,2020-08-01 19:07:09,-46.60,168.33,51.96,79,100,12.91
...,...,...,...,...,...,...,...,...,...,...
541,541,Kaeo,NZ,2020-08-01 19:09:37,-35.10,173.78,59.00,3,100,7.00
546,546,Christchurch,NZ,2020-08-01 19:07:30,-43.53,172.63,53.01,71,100,9.17
556,556,Verkhnetulomskiy,RU,2020-08-01 19:11:31,68.61,31.80,50.00,93,75,2.24
558,558,Masterton,NZ,2020-08-01 19:11:32,-40.96,175.66,45.00,84,91,4.34


In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
14,Hobart,AU,50.00,-42.88,147.33,
16,Masty,BY,57.79,53.41,24.54,
20,New Norfolk,AU,48.99,-42.78,147.06,
21,Mount Gambier,AU,46.40,-37.83,140.77,
22,Bluff,NZ,51.96,-46.60,168.33,
23,Grahamstown,ZA,45.34,-33.30,26.53,
26,Mar Del Plata,AR,57.00,-38.00,-57.56,
27,Havoysund,NO,46.40,71.00,24.66,
28,Busselton,AU,51.01,-33.65,115.33,
30,Husavik,IS,59.00,66.04,-17.34,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [13]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    
    
    
    
    
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
14,Hobart,AU,50.00,-42.88,147.33,St Ives Apartments
16,Masty,BY,57.79,53.41,24.54,Gostinitsa Masty
20,New Norfolk,AU,48.99,-42.78,147.06,The Shingles Riverside Cottages
21,Mount Gambier,AU,46.40,-37.83,140.77,Southgate Motel
22,Bluff,NZ,51.96,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
...,...,...,...,...,...,...
541,Kaeo,NZ,59.00,-35.10,173.78,Bushido Lodge
546,Christchurch,NZ,53.01,-43.53,172.63,The George
556,Verkhnetulomskiy,RU,50.00,68.61,31.80,Отель Уют
558,Masterton,NZ,45.00,-40.96,175.66,Copthorne Hotel & Resort Solway Park Wairarapa


In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))